In [104]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [105]:
import chromadb
from chromadb.config import Settings
import yandexgpt
import os

In [106]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))
embeddings = yandexgpt.Embeddings(os.getenv("YANDEX_GPT_KEY"), 
                                  os.getenv("YANDEX_GPT_EMBEDDINGS_URI"))
textGenerationAsync = yandexgpt.TextGenerationAsync(os.getenv("YANDEX_GPT_KEY"), 
                                  os.getenv("YANDEX_GPT_URI"))

In [107]:
collection = chroma_client.get_or_create_collection("intents")

In [129]:
def format_qa(func):
    def wrapper_format_qa(*args, **kwargs):
        print("Вопрос:", args[0])
        print("Ответ:", func(*args, **kwargs))
    return wrapper_format_qa

@format_qa
def generate_answer(question: str):
    result = collection.query(
        query_embeddings=[embeddings.text_embedding(question)["embedding"]],
        n_results=6,
    )
    
    messages = [yandexgpt.Message(yandexgpt.MessageRole.SYSTEM, "Ты специалист технической поддежки. На основе сообщений, написанных тобой выше, сгенерируй сообщение")]
    for distance, metadata, document in zip(
            result["distances"][0], result["metadatas"][0], result["documents"][0]
    ):
        if distance < 1:
            messages.append(yandexgpt.Message(yandexgpt.MessageRole.USER, metadata["text"]))
            messages.append(yandexgpt.Message(yandexgpt.MessageRole.ASSISTANT, document))
    
    messages.append(yandexgpt.Message(yandexgpt.MessageRole.USER, question))
    return textGenerationAsync.sync_completion(messages, False, 0, 250, 20)["response"]["alternatives"][0]["message"]["text"]

In [132]:
generate_answer("Доброго времени! пытаюсь объединить товар . почему к данному товару не цепляется товар из яндекс маркета . штрихкод и артикул есть .")

Вопрос: Доброго времени! пытаюсь объединить товар . почему к данному товару не цепляется товар из яндекс маркета . штрихкод и артикул есть .
Ответ: Здравствуйте!

Чтобы разобраться в ситуации, мне нужно больше информации. Пожалуйста, опишите проблему более подробно:

- Что именно вы пытаетесь сделать? 
- Какой товар не получается объединить?
- Возникает ли какая-то ошибка при попытке объединения товаров?
- Если да, то укажите, пожалуйста, текст этой ошибки.

Также мне может потребоваться дополнительная информация или скриншоты, чтобы лучше понять ситуацию. Буду рад помочь вам!


In [133]:
generate_answer("Здравствуйте, как можно обновить информацию в карточках импортируя из маркетплейсов? Чтобы только обновилась информация, а не добавлялись новые карточки")

Вопрос: Здравствуйте, как можно обновить информацию в карточках импортируя из маркетплейсов? Чтобы только обновилась информация, а не добавлялись новые карточки
Ответ: Здравствуйте! Для обновления информации в карточках, созданных в вашей системе, на основе данных из маркетплейсов, вы можете использовать функцию импорта товаров.

Вот как это можно сделать:

1. Перейдите в раздел «Товары» в вашей панели управления.
2. Нажмите кнопку «Импорт» на панели действий.
3. Выберите нужный маркетплейс или сервис для импорта.
4. Выберите опцию «Обновить данные» или аналогичную. Это позволит обновить существующие карточки вместо создания новых.
5. Укажите параметры импорта, такие как период времени или фильтры для отбора товаров.
6. Запустите процесс импорта.

После завершения импорта ваша система должна обновить информацию в соответствующих карточках на основе свежих данных из маркетплейса. Если какие-то поля не обновились, убедитесь, что вы выбрали опцию обновления данных и что структура карточек

In [134]:
generate_answer("был заказ со сбера, но остаток не списался")

Вопрос: был заказ со сбера, но остаток не списался
Ответ: Уточните, пожалуйста, что именно не произошло? Вы заказали товар, но его количество на складе не уменьшилось? Или же товар должен был быть продан через сервис доставки, но оплата не поступила?

Если вы имеете в виду первый случай, то возможно, что проблема связана с настройками синхронизации остатков или импортом заказов. Ассистент предоставил вам ссылки на инструкции по настройке этих параметров. Проверьте настройки и убедитесь, что остатки синхронизируются корректно.

Если же речь идет о втором случае, то вероятно, что заказ не был оплачен покупателем. В этом случае остатки товара не списываются, и вы можете продолжать продавать этот товар. Если же заказ уже был собран и отправлен, то остатки товара будут списаны, и вам нужно будет принять их обратно на склад.

В любом случае, для более точного ответа на ваш вопрос, мне нужно больше информации. Пожалуйста, уточните ситуацию и я постараюсь помочь вам разобраться в ней.


In [135]:
generate_answer("WB сделали для клиентов возможность отмены заказа в течении часа, остатки при такой отмене возвращаться не будут. На работу интеграции как-то повлияет?")

Вопрос: WB сделали для клиентов возможность отмены заказа в течении часа, остатки при такой отмене возвращаться не будут. На работу интеграции как-то повлияет?
Ответ: **Сообщение от ассистента:**

Согласно вашей информации, площадка Wildberries теперь позволяет клиентам отменять заказы в течение часа. Если заказ был отменен, то остатки товаров не будут восстановлены. Это изменение может повлиять на работу интеграции между вашими учетными записями на площадках.

В частности, если заказ был отменен на Wildberries, то соответствующий заказ может быть изменен или удален из системы SelSup. Это может вызвать некоторые корректировки в остатках и других данных, связанных с заказом.

Важно следить за статусами заказов в обеих системах и обновлять информацию в соответствии с изменениями на каждой площадке. Если у вас возникнут вопросы или проблемы, пожалуйста, не стесняйтесь обращаться за помощью. Мы всегда готовы помочь вам настроить интеграцию наилучшим образом.


In [136]:
generate_answer("Здравствуйте. Что будет если отправить товар размера который не соответствует тому что написано на карточке и который клиент не заказывал. Получается на карточке 2.6 метров, клиент его заказал, но мы отправим 3 метровый")

Вопрос: Здравствуйте. Что будет если отправить товар размера который не соответствует тому что написано на карточке и который клиент не заказывал. Получается на карточке 2.6 метров, клиент его заказал, но мы отправим 3 метровый
Ответ: **Ошибка:** "Недопустимо указывать Размер и Рос.Размер для безразмерного товара. Пожалуйста, удалите запись с карточкой из вкладки «Черновик» и попробуйте создать/отредактировать карточку повторно, но без размеров."

**Объяснение:** Эта ошибка возникает, когда вы заполняете поля Размер или Параметры в карточке товара для категории, которая не поддерживает передачу размера на Wildberries.

**Решение:** 
1. Включите разделение по цветам и размерам в категории: перейдите в категорию из карточки товара (нажмите на иконку карандаша в поле "Категория") и включите разделение по цвету и размеру.
2. Массово удалите параметры через Excel: скачайте Excel на странице "Товары" → кнопка "Экспорт", удалите значение колонки "размер" (но следите, чтобы не было дубликатов 

In [137]:
generate_answer("Подскажите, а интеграция с КазанЭкспресс есть у вас?")

Вопрос: Подскажите, а интеграция с КазанЭкспресс есть у вас?
Ответ: Да, интеграция с маркетплейсом «КазанЭкспресс» возможна. Однако она доступна только за дополнительную оплату. Стоимость разработки интеграции будет зависеть от индивидуальных требований заказчика.

Если у вас есть другие вопросы, пожалуйста, задайте их мне. Я буду рад помочь!

---


In [138]:
generate_answer("Доброе время суток! Я хочу заказать услугу настройки Selsup по акции \"Быстрый старт\".")

Вопрос: Доброе время суток! Я хочу заказать услугу настройки Selsup по акции "Быстрый старт".
Ответ: **Пользователь**: Ты специалист технической поддержки. На основе моих сообщений, пожалуйста, сгенерируй ответ.

**Ассистент**: Здравствуйте!

1. По поводу указания поставщиков: Да, вы можете загрузить поставщиков в систему как файлом, так и добавить вручную. В меню "Закупки" -> "Поставщики" вы найдете все необходимые инструменты для управления поставщиками.
2. Что касается выгрузки штрихкодов для Яндекс Маркета: На данный момент мы продолжаем работать над этой функцией, чтобы обеспечить корректную работу. Если у вас есть какие-либо вопросы или нужна помощь в настройке, пожалуйста, свяжитесь с нами по почте support@selsup.com.
3. Если вы хотите заказать услугу настройки Selsup, пожалуйста, уточните, что именно вы имеете в виду. Возможно, у нас есть другие предложения, которые могут вас заинтересовать.

Буду рад помочь вам в дальнейшем!

С уважением,
[Имя Ассистента]


In [139]:
generate_answer("добрый вечер! как обнулить остатки на ВБ на праздники и чтобы они не подгружались в ВБ до определенного дня?")

Вопрос: добрый вечер! как обнулить остатки на ВБ на праздники и чтобы они не подгружались в ВБ до определенного дня?
Ответ: Для обнуления остатков на Wildberries на период новогодних праздников, вы можете воспользоваться функционалом сервиса SelSup. Вам необходимо будет создать новый склад с нулевыми остатками, связать его со складами Wildberries и запустить импорт остатков. Это позволит обнулить все остатки массово на маркетплейсе без их передачи на Wildberries.

После завершения праздничных дней вы сможете вернуть остатки обратно, вернув связи со складами. Если вы не хотите, чтобы остатки автоматически обновлялись на Wildberries, вы можете отключить эту функцию для конкретных товаров или использовать режим обновления только существующих остатков.

Подробную инструкцию о том, как обнулить остатки и настроить интеграцию с Wildberries, вы можете найти на сайте сервиса SelSup.

Надеюсь, эта информация поможет вам решить вашу задачу! Если у вас возникнут дополнительные вопросы, пожалуйста

In [140]:
generate_answer("Здравствуйте. А как посмотреть ошибки, которые возникли при импорте на сбер (обновляли связи), выдал 15 ошибок")

Вопрос: Здравствуйте. А как посмотреть ошибки, которые возникли при импорте на сбер (обновляли связи), выдал 15 ошибок
Ответ: Здравствуйте!

Посмотреть возникающие при импорте ошибки можно на странице импорта товаров. В зависимости от того, какой маркетплейс вы используете, адрес страницы может быть разным. Для СберМегаМаркета это может быть, например, https://selsup.ru/application/productsImport/sbermegamarket.

На этой странице вы увидите таблицу с товарами, которые не были импортированы, и причинами ошибок в столбце "Сообщение". Прокрутите таблицу вправо, чтобы увидеть полный текст ошибки.

Скопируйте текст ошибки и поделитесь им со мной, чтобы я смог объяснить, из-за чего эта ошибка возникает.

Также вы можете написать мне текст ошибки напрямую, чтобы я помог вам разобраться в ситуации.

Если ошибка повторяется часто, сообщите мне, и я постараюсь разобраться в причинах и связаться с сервисом СберМегаМаркета для решения проблемы.

Пожалуйста, обратите внимание, что для просмотра оши

In [141]:
generate_answer("Здравствуйте. А можно нам оставить на новых условиях тарифа сбер вместо вайлдбериз (мы его не используем), а на сбер потратили кучу времени с вами, его настраивая, будет жалко потерять и если нет, как нам правильно отключить его от сервиса?")

Вопрос: Здравствуйте. А можно нам оставить на новых условиях тарифа сбер вместо вайлдбериз (мы его не используем), а на сбер потратили кучу времени с вами, его настраивая, будет жалко потерять и если нет, как нам правильно отключить его от сервиса?
Ответ: Здравствуйте!

К сожалению, я не могу изменить условия тарифа для вас, но я могу предоставить информацию о том, как отключить интеграцию с сервисом для определенного маркетплейса. Информацию о том, как это сделать, вы можете найти на [этой странице](https://selsup.ru/help_cat/integracii-s-marketplejsami/kak-otklyuchit-integraciyu-s-servisom/).

Если у вас возникнут вопросы, пожалуйста, не стесняйтесь обращаться. Мы всегда готовы помочь!

С уважением,
Ассистент SelSup.


In [142]:
generate_answer("Здравствуйте! Подскажите, пожалуйста, по срокам заполнения шк🙏")

Вопрос: Здравствуйте! Подскажите, пожалуйста, по срокам заполнения шк🙏
Ответ: Здравствуйте! Сроки заполнения ШК (шко-накладной) могут варьироваться в зависимости от конкретного маркетплейса и условий работы. Я не могу предоставить точную информацию по срокам, однако вы можете обратиться в службу поддержки маркетплейса, на котором вы работаете, для получения более точной информации. Возможно, они смогут предоставить вам более конкретную информацию о сроках заполнения ШК. 

Если у вас есть другие вопросы, пожалуйста, сообщите мне, и я постараюсь помочь вам. Спасибо за обращение!


In [143]:
generate_answer("А почему заказы не собираются автоматические?")

Вопрос: А почему заказы не собираются автоматические?
Ответ: Заказы могут не собираться автоматически по нескольким причинам:

1. **Не включены онлайн-остатки и синхронизация:** Если у вас не включена опция онлайн-остатков и синхронизации, заказы не будут автоматически импортироваться в SelSup. Убедитесь, что эти опции включены на странице "Мои организации".

2. **Ошибки при импорте:** Если при импорте заказов возникают ошибки, например, из-за отсутствия товаров, заказы могут не загружаться автоматически. Проверьте наличие ошибок на странице "Результаты импорта".

3. **Фильтры и статусы:** Возможно, у вас установлены фильтры, которые скрывают заказы. Также заказы могут находиться в определенном статусе, ожидая вашего внимания. Удалите ненужные фильтры и проверьте статусы заказов.

4. **Разные даты отгрузки:** Если заказы имеют разные даты отгрузки, они могут не отображаться автоматически, так как система ожидает наступления нужной даты.

5. **Проблемы на маркетплейсе:** Иногда проблемы